In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from splinter import Browser
import warnings
warnings.filterwarnings('ignore')

#pointing to the directory where chromedriver exists
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser("chrome", **executable_path, headless = False)

In [3]:
#url = 'https://www.remax.ca/ab/calgary-real-estate'
province_abb = input("Please enter the abbreviation of your province: ")
province_name = province_abb.lower()
city_name_input = input("Please enter city: ")
city_name = city_name_input.lower()
base_url = f'https://www.remax.ca/{province_name}/{city_name}-real-estate'
print(base_url)

Please enter the abbreviation of your province: ab
Please enter city: calgary
https://www.remax.ca/ab/calgary-real-estate


In [4]:
house_address = []
house_details = []


base_url = f'https://www.remax.ca/{province_name}/{city_name}-real-estate?page='
urls = [base_url + str(x) for x in range(1,2)]


for url in urls:
    
    #browser.visit(url)
    time.sleep(5)
    
    # HTML Object
    #html = browser.html

    # Parse HTML with Beautiful Soup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    addresses = soup.find_all('div', class_='left-content flex-one')
    for address in addresses:
        house_address.append(address.text)
        
    details = soup.find_all('div', class_='property-details')
    for detail in details:
        house_details.append(detail.text)
#print(soup)
  

In [13]:
#house_address
#house_details

In [5]:
address_df = pd.DataFrame(house_address)

new_df = address_df[0].str.split(' ', 2, expand=True)

del new_df[0]
new_df.head()

,1,2
0,"$489,000","140 CITADEL CREST CIR NW, Calgary, AB, T3G 4G3"
1,"$351,999","79 GALBRAITH DR SW, Calgary, AB, T3E 4Z5"
2,"$449,999","275 TARAVISTA ST NE, Calgary, AB, T3J 4S5"
3,"$469,900","25 DOUGLASBANK RISE SE, Calgary, AB, T2Z 2C5"
4,"$419,900","111 PANORAMA HILLS PL NW, Calgary, AB, T3K 4R9"


In [6]:
final_df = new_df[2].str.split(', Calgary, AB, ', expand=True)
# final_df = new_df[2].str.split(', Toronto, ON, ', expand=True)
final_df.head()

,0,1
0,140 CITADEL CREST CIR NW,T3G 4G3
1,79 GALBRAITH DR SW,T3E 4Z5
2,275 TARAVISTA ST NE,T3J 4S5
3,25 DOUGLASBANK RISE SE,T2Z 2C5
4,111 PANORAMA HILLS PL NW,T3K 4R9


In [7]:
df_add = pd.concat([new_df, final_df], axis=1)
del df_add[2]
df_add.columns = ["price", "address", "postal_code"]
df_add.head(20)

,price,address,postal_code
0,"$489,000",140 CITADEL CREST CIR NW,T3G 4G3
1,"$351,999",79 GALBRAITH DR SW,T3E 4Z5
2,"$449,999",275 TARAVISTA ST NE,T3J 4S5
3,"$469,900",25 DOUGLASBANK RISE SE,T2Z 2C5
4,"$419,900",111 PANORAMA HILLS PL NW,T3K 4R9
5,"$725,000",3702 21 AVE SW,T3E 7Z1
6,"$660,000",88 COUGARSTONE MANOR SW,T3H 5N5
7,"$839,900",19 WESTRIDGE CRES SW,T3H 5C9
8,"$360,000",135 COVEWOOD CLOSE NE,T3K 4Z8
9,"$799,900",5620 LODGE CRES SW,T3E 5Y7


In [8]:
details = pd.DataFrame(house_details)

details_df = details[0].str.split('|', expand=True)
details_df

del details_df[2]

details_df.columns = ["bedrooms", "bath", "property_type"]
details_df.head()

,bedrooms,bath,property_type
0,3 bed,2 + 1 bath,house
1,N/A bed,N/A bath,house
2,5 bed,2 + 1 bath,house
3,4 bed,2 + 1 bath,house
4,3 bed,2 bath,house


In [9]:
# toronto_df = pd.concat([df_add, details_df], axis=1)
# toronto_df.head()


calgary_df = pd.concat([df_add, details_df], axis=1)
calgary_df.head()

,price,address,postal_code,bedrooms,bath,property_type
0,"$489,000",140 CITADEL CREST CIR NW,T3G 4G3,3 bed,2 + 1 bath,house
1,"$351,999",79 GALBRAITH DR SW,T3E 4Z5,N/A bed,N/A bath,house
2,"$449,999",275 TARAVISTA ST NE,T3J 4S5,5 bed,2 + 1 bath,house
3,"$469,900",25 DOUGLASBANK RISE SE,T2Z 2C5,4 bed,2 + 1 bath,house
4,"$419,900",111 PANORAMA HILLS PL NW,T3K 4R9,3 bed,2 bath,house


----

### **Testing**

# Testing Walkscore Score

In [32]:
postal_code = input("Please enter your postal code: ")
if len(postal_code) != 7:
    print ("Error! Only 7 characters allowed! Please use the correct format")



Please enter your postal code: T2V 2L4


In [33]:
postal_code = postal_code.replace(" ", "%20")

In [52]:
url = "https://www.walkscore.com/score/" + str(postal_code)
print(url)

https://www.walkscore.com/score/T2V%202L4


# Code Starts here

In [10]:
post_code_list = []

for i in calgary_df["postal_code"]:
    post_code_list.append(i)
    
post_code_list_3 = post_code_list[0:2]
post_code_list_3

['T3G 4G3', 'T3E 4Z5']

In [11]:
score_list = []

for i in post_code_list:
    try:
        postal_code = i.replace(" ", "%20")
        url_score = "https://www.walkscore.com/score/" + str(postal_code)

        #executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
        #browser = Browser("chrome", **executable_path, headless = True)

        #browser.visit(url)
        time.sleep(2)

        # HTML Object
        #code_html = browser.html

        # Parse HTML with Beautiful Soup
        response = requests.get(url_score)
        code_soup = BeautifulSoup(response.text, 'html.parser')

        walk_score1 = code_soup.find_all('div', class_='clearfix score-div')
        time.sleep(1)

        walk_score_list = []

        for j in walk_score1:
            walk_score_list.append(j.img['alt'])

            scores = []

            for k in walk_score_list:
                number = k.split(" ")[0]
                scores.append(number)


        score_list.append(scores)
        
    except AttributeError:
        empty = []
        score_list.append(empty)
        
#         print("None")
        

            
        

score_list = []

for i in post_code_list_3:
    try:
        #postal_code = i.replace(" ", "%20")
        #url_score = "https://www.walkscore.com/score/" + str(postal_code)

        #executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
        time.sleep(2)
        browser = Browser('firefox', headless = False)
        
        url_walk = 'https://www.walkscore.com/score/'
        browser.visit(url_walk)
        browser.find_by_id("addrbar-street").fill(i)
        browser.click_link_by_partial_text("Go")
        time.sleep(3)
        
        

        # HTML Object
        code_html = browser.html
        browser.quit()
        

        # Parse HTML with Beautiful Soup
        response = requests.get(code_html)
        code_soup = BeautifulSoup(response.text, 'html.parser')

        walk_score1 = code_soup.find_all('div', class_='clearfix score-div')
        time.sleep(1)

        walk_score_list = []

        for j in walk_score1:
            walk_score_list.append(j.img['alt'])

            scores = []

            for k in walk_score_list:
                number = k.split(" ")[0]
                scores.append(number)


        score_list.append(scores)
        
        
    except AttributeError:
        empty = []
        score_list.append(empty)
        
        


In [12]:
print(score_list)

[['23', '38', '65'], ['43', '47', '67'], ['10', '49', '62'], ['22', '41', '65'], ['37', '48', '71'], ['51', '60'], ['1', '26', '36'], ['35', '37', '72'], ['9', '30', '45'], ['12', '37', '53'], ['45', '46', '55'], ['30', '41', '70'], ['86', '69', '99'], ['3', '27', '37'], ['17', '44', '48'], ['94', '76', '94'], ['1', '35', '61'], ['9', '36', '56'], ['49', '31', '62'], ['91', '67', '97'], ['23', '38', '65'], ['43', '47', '67'], ['10', '49', '62'], ['22', '41', '65'], ['37', '48', '71'], ['51', '60'], ['1', '26', '36'], ['35', '37', '72'], ['9', '30', '45'], ['12', '37', '53'], ['45', '46', '55'], ['30', '41', '70'], ['86', '69', '99'], ['3', '27', '37'], ['17', '44', '48'], ['94', '76', '94'], ['1', '35', '61'], ['9', '36', '56'], ['49', '31', '62'], ['91', '67', '97']]


In [13]:
len(score_list)

40

In [14]:
len(post_code_list)

40

In [15]:
score_df = pd.DataFrame(score_list, columns=['walk_score', 
                                             'transit_score', 
                                             'bike_score'])
score_df.head()

,walk_score,transit_score,bike_score
0,23,38,65
1,43,47,67
2,10,49,62
3,22,41,65
4,37,48,71


In [39]:
# copy_dd_test = data_dump1_test.copy()
# copy_dd_test.head()

In [16]:
postal_code_df = pd.DataFrame(post_code_list, columns=['postal_code'])
merged_df = pd.concat([postal_code_df, score_df], axis=1)
merged_df.head()

,postal_code,walk_score,transit_score,bike_score
0,T3G 4G3,23,38,65
1,T3E 4Z5,43,47,67
2,T3J 4S5,10,49,62
3,T2Z 2C5,22,41,65
4,T3K 4R9,37,48,71



new_df = pd.concat([calgary_df, score_df], axis=1)
new_df.head()

In [17]:
calgary_df.head()

,price,address,postal_code,bedrooms,bath,property_type
0,"$489,000",140 CITADEL CREST CIR NW,T3G 4G3,3 bed,2 + 1 bath,house
1,"$351,999",79 GALBRAITH DR SW,T3E 4Z5,N/A bed,N/A bath,house
2,"$449,999",275 TARAVISTA ST NE,T3J 4S5,5 bed,2 + 1 bath,house
3,"$469,900",25 DOUGLASBANK RISE SE,T2Z 2C5,4 bed,2 + 1 bath,house
4,"$419,900",111 PANORAMA HILLS PL NW,T3K 4R9,3 bed,2 bath,house


https://www.walkscore.com/score/t2m%202c7

In [18]:
from sqlalchemy import create_engine

In [19]:
rds_connection_string = "postgres:1@localhost:5432/realestate_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [25]:
engine.table_names()

['score_df', 'calgary_df']

In [29]:
calgary_df.to_sql(name= "calgary_df", 
                      con=engine, 
                      if_exists="append", 
                      index=False)